### Data Preprocessing
- Extract frames from left.mp4.
- Create det.txt similar to MOT17 (transform left_people.json into the format of MOT17 FRCNN).
- Organize the folder structure of StreetAware to match that of MOT17 for applying bot-sort.

### extract frames from video

In [ ]:
!ffmpeg -i left.mp4  -vframes 1000 img_left/%06d.jpg

In [ ]:
#%ls img1/*.jpg | wc -l
# a total of 40395 frames extracted

40395


In [ ]:
import json
with open('left_people.json', 'r') as file:
    left_people = json.load(file)

In [ ]:
print(len(left_people))

40395


In [ ]:
import json
with open('right_people.json', 'r') as file:
    right_people = json.load(file)
print(len(right_people))

40395


### Remove the empty rows from left_people.json (which don't have any objects detected) and the corresponding frames
- remove the empty rows in left_people.json
- remove the corresponding frames and reset the frame names

In [ ]:
import json
import os

# Load JSON data
json_file_path = 'left_people.json'
with open(json_file_path, 'r') as file:
    left_people = json.load(file)

# Directory containing frames
img_dir = 'img_left/'
available_images = sorted(os.listdir(img_dir))
num_images = len(available_images)

# Identify indices to remove
indices_to_remove = [i for i, person in enumerate(left_people) if person is None or i >= num_images]

# Filter out the unnecessary entries
left_people_cleaned = [person for i, person in enumerate(left_people) if i not in indices_to_remove and i < num_images]

# Save the cleaned JSON data
with open('left_people_cleaned.json', 'w') as file:
    json.dump(left_people_cleaned, file)

# Delete corresponding frames not needed or out of sync
for index in sorted(indices_to_remove, reverse=True):
    if index < num_images:
        img_filename = f"{index + 1:06d}.jpg"
        img_path = os.path.join(img_dir, img_filename)
        if os.path.exists(img_path):
            os.remove(img_path)
            print(f"Deleted {img_path}")

remaining_images = sorted(os.listdir(img_dir))
new_index = 1
for old_name in remaining_images:
    new_name = f"{new_index:06d}.jpg"
    old_path = os.path.join(img_dir, old_name)
    new_path = os.path.join(img_dir, new_name)
    if old_path != new_path:
        os.rename(old_path, new_path)
        print(f"Renamed {old_path} to {new_path}")
    new_index += 1


### convert the left_people_cleaned.json to FCRNN format as in MOT17

In [ ]:
import csv
with open('left_people_cleaned_mot.txt', 'w', newline='') as txtfile:
    writer = csv.writer(txtfile, delimiter=',')
    for frame_index, frame_data in enumerate(left_people_cleaned):
        if frame_data:
            for detection_index, box in enumerate(frame_data.get('boxes', [])):
                try:
                    bb_top_left = box[0]
                    bb_bottom_right = box[1]
                    bb_left = bb_top_left[0]
                    bb_top = bb_top_left[1]
                    bb_width = bb_bottom_right[0] - bb_left
                    bb_height = bb_bottom_right[1] - bb_top
                    conf = frame_data['scores'][detection_index] if 'scores' in frame_data and len(frame_data['scores']) > detection_index else -1
                    writer.writerow([
                        frame_index + 1,   # <frame>
                        -1,  # <identity id>
                        bb_left, bb_top, bb_width, bb_height,  # <boundingbox left>,<boundingbox top>,<boundingbox width>,<boundingbox height>
                        conf  # <confidence score>
                    ])
                except IndexError:
                    print(f"Error processing data for frame {frame_index+1}, detection {detection_index+1}")
        else:
            writer.writerow([frame_index + 1, -1, -1, -1, -1, -1, -1])

In [ ]:
import csv
with open('left_people_cleaned_mot.txt', 'r') as file:
    reader = csv.reader(file)
    left_people_cleaned_mot = list(reader)

In [ ]:
import configparser

# Create a configparser object
config = configparser.ConfigParser()

# Add the sequence section
config['Sequence'] = {
    'name': 'sensor_1_left',
    'imDir': 'img1',
    'frameRate': 14.40,
    'seqLength': 39617,
    'imWidth': 2592,
    'imHeight': 1944,
    'imExt': '.jpg'
}

with open('seqinfo.ini', 'w') as configfile:
    config.write(configfile)


### create a sample

In [ ]:
import os
import shutil

# Set the source and destination directories
src_img_dir = 'img_left'
dst_img_dir = 'img_left_sample'
src_det_file = 'left_people_cleaned_mot.txt'
dst_det_file = 'det_left_sample/det.txt'

# Create destination directories if they don't exist
os.makedirs(dst_img_dir, exist_ok=True)
os.makedirs(os.path.dirname(dst_det_file), exist_ok=True)

# Copy the first 900 images
max_frame = 900
for frame_num in range(1, max_frame + 1):
    src_img_path = os.path.join(src_img_dir, f"{frame_num:06d}.jpg")
    dst_img_path = os.path.join(dst_img_dir, f"{frame_num:06d}.jpg")
    if os.path.exists(src_img_path):
        shutil.copy(src_img_path, dst_img_path)

# Filter det.txt for the first 900 frames
with open(src_det_file, 'r') as src_file, open(dst_det_file, 'w') as dst_file:
    for line in src_file:
        frame_id = int(line.split(',')[0])
        if frame_id <= max_frame:
            dst_file.write(line)
        elif frame_id > max_frame:
            break
